<a href="https://colab.research.google.com/github/dalismo/Finance_News_Sentiment/blob/ai%2Fall_covid_stage_api_pull_run_machine/2_4_post_vax_prediction_counts_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("StopWords").getOrCreate()
spark = SparkSession.builder.appName("Fin-sent-count-vector").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/sentiment-train.csv'
# df = pd.read_csv(url,encoding='latin-1', header=None)
# url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/yelp_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sentiment-train.csv"), sep=",", header=None)



# Show DataFrame
df.show(5, truncate=False)


+--------+-----------------------------------------------------------------------------------------------------+
|_c0     |_c1                                                                                                  |
+--------+-----------------------------------------------------------------------------------------------------+
|Positive|Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                              |
|Positive|Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                     |
|Positive|RSX: Russian Equities Are Still Undervalued                                                          |
|Positive|XLB: Hold Your Position                                                                              |
|Positive|Musgrave Minerals broad, near-surface high-grade results open up further Cue gold discovery potential|
+--------+--------------------------------------------------------------------------------------

In [5]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['_c1']))
data_df.show()

+--------+--------------------+------+
|     _c0|                 _c1|length|
+--------+--------------------+------+
|Positive|Autodesk Is Down ...|    71|
|Positive|Blue Shield of Ca...|    80|
|Positive|RSX: Russian Equi...|    43|
|Positive|XLB: Hold Your Po...|    23|
|Positive|Musgrave Minerals...|   101|
|Positive|ViacomCBS to rest...|    49|
|Positive|Oil climbs to one...|    51|
|Negative|Australian regula...|    67|
|Positive|The Wall Street J...|    90|
|Positive|Ora Banda Mining ...|    50|
|Positive|Google Pay Plans ...|    43|
|Negative|Toyota, Honda Ang...|    69|
| Neutral|ZY Investor Alert...|    44|
| Neutral|Calima Energy spu...|    58|
|Positive|These Marijuana S...|    56|
|Positive|Moderna ETFs to R...|    61|
| Neutral|MacroGenics Annou...|    72|
| Neutral|Zymeworks Announc...|   173|
| Neutral|BioXcel Therapeut...|   130|
| Neutral|Merck to Present ...|    80|
+--------+--------------------+------+
only showing top 20 rows



In [6]:
df2 = data_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
df2.printSchema()
df2.show(20, truncate = False)


root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|class   |text                                                                                                                                                                         |length|
+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Positive|Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                                                                                                      |71    |
|Positive|Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                        

In [7]:
df2.createOrReplaceTempView('temp')
df3 = spark.sql('select case class when "Positive" then 1.0  when "Negative" then 2.0 else 0 end as label, text from temp')
df3.show(20, truncate = False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|text                                                                                                                                                                         |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1.0  |Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                                                                                                      |
|1.0  |Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                                                                                             |
|1.0  |RSX: Russian Equities Are Still Undervalued                                        

In [8]:
from pyspark.ml.feature import  Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(df3)
wordsData.show()

+-----+--------------------+--------------------+
|label|                text|               words|
+-----+--------------------+--------------------+
|  1.0|Autodesk Is Down ...|[autodesk, is, do...|
|  1.0|Blue Shield of Ca...|[blue, shield, of...|
|  1.0|RSX: Russian Equi...|[rsx:, russian, e...|
|  1.0|XLB: Hold Your Po...|[xlb:, hold, your...|
|  1.0|Musgrave Minerals...|[musgrave, minera...|
|  1.0|ViacomCBS to rest...|[viacomcbs, to, r...|
|  1.0|Oil climbs to one...|[oil, climbs, to,...|
|  2.0|Australian regula...|[australian, regu...|
|  1.0|The Wall Street J...|[the, wall, stree...|
|  1.0|Ora Banda Mining ...|[ora, banda, mini...|
|  1.0|Google Pay Plans ...|[google, pay, pla...|
|  2.0|Toyota, Honda Ang...|[toyota,, honda, ...|
|  0.0|ZY Investor Alert...|[zy, investor, al...|
|  0.0|Calima Energy spu...|[calima, energy, ...|
|  1.0|These Marijuana S...|[these, marijuana...|
|  1.0|Moderna ETFs to R...|[moderna, etfs, t...|
|  0.0|MacroGenics Annou...|[macrogenics, ann...|


In [9]:
from pyspark.ml.feature import CountVectorizer
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
model = count.fit(wordsData)
featurizedData = model.transform(wordsData)
featurizedData.show()

+-----+--------------------+--------------------+--------------------+
|label|                text|               words|         rawFeatures|
+-----+--------------------+--------------------+--------------------+
|  1.0|Autodesk Is Down ...|[autodesk, is, do...|(11788,[6,9,19,37...|
|  1.0|Blue Shield of Ca...|[blue, shield, of...|(11788,[2,7,177,5...|
|  1.0|RSX: Russian Equi...|[rsx:, russian, e...|(11788,[48,262,63...|
|  1.0|XLB: Hold Your Po...|[xlb:, hold, your...|(11788,[158,335,1...|
|  1.0|Musgrave Minerals...|[musgrave, minera...|(11788,[26,49,89,...|
|  1.0|ViacomCBS to rest...|[viacomcbs, to, r...|(11788,[0,34,1891...|
|  1.0|Oil climbs to one...|[oil, climbs, to,...|(11788,[0,7,100,1...|
|  2.0|Australian regula...|[australian, regu...|(11788,[5,175,179...|
|  1.0|The Wall Street J...|[the, wall, stree...|(11788,[1,7,292,4...|
|  1.0|Ora Banda Mining ...|[ora, banda, mini...|(11788,[0,5,72,32...|
|  1.0|Google Pay Plans ...|[google, pay, pla...|(11788,[75,217,26...|
|  2.0

In [10]:
from pyspark.ml.feature import  IDF
#IDF down-weighs features which appear frequently in a corpus. This generally improves performance when using text as features since most frequent, and hence less important words, get down-weighed.
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "features").show()  
# We want only the label and features columns for our machine learning models

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(11788,[6,9,19,37...|
|  1.0|(11788,[2,7,177,5...|
|  1.0|(11788,[48,262,63...|
|  1.0|(11788,[158,335,1...|
|  1.0|(11788,[26,49,89,...|
|  1.0|(11788,[0,34,1891...|
|  1.0|(11788,[0,7,100,1...|
|  2.0|(11788,[5,175,179...|
|  1.0|(11788,[1,7,292,4...|
|  1.0|(11788,[0,5,72,32...|
|  1.0|(11788,[75,217,26...|
|  2.0|(11788,[2,65,230,...|
|  0.0|(11788,[43,44,63,...|
|  0.0|(11788,[10,83,338...|
|  1.0|(11788,[0,3,13,18...|
|  1.0|(11788,[0,7,172,2...|
|  0.0|(11788,[11,12,13,...|
|  0.0|(11788,[1,3,5,11,...|
|  0.0|(11788,[0,2,3,12,...|
|  0.0|(11788,[0,4,12,13...|
+-----+--------------------+
only showing top 20 rows



In [11]:
seed = 0  # set seed for reproducibility
trainDF, testDF = rescaledData.randomSplit([0.8,0.2],seed)

In [12]:
trainDF.count()

3965

In [13]:
trainDF.show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
testDF.count()

1035

In [15]:
testDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+
|label|                text|               words|         rawFeatures|            features|
+-----+--------------------+--------------------+--------------------+--------------------+
|  0.0|180 Life Sciences...|[180, life, scien...|(11788,[10,12,197...|(11788,[10,12,197...|
|  0.0|1847 Goedeker Iss...|[1847, goedeker, ...|(11788,[45,87,474...|(11788,[45,87,474...|
|  0.0|3 Reasons to Buy ...|[3, reasons, to, ...|(11788,[0,4,23,54...|(11788,[0,4,23,54...|
|  0.0|3 Things to Watch...|[3, things, to, w...|(11788,[0,1,3,8,3...|(11788,[0,1,3,8,3...|
|  0.0|3 Upcoming Earnin...|[3, upcoming, ear...|(11788,[0,19,54,6...|(11788,[0,19,54,6...|
|  0.0|4 Diversified Che...|[4, diversified, ...|(11788,[0,5,18,17...|(11788,[0,5,18,17...|
|  0.0|4 Outstanding Sto...|[4, outstanding, ...|(11788,[0,18,20,2...|(11788,[0,18,20,2...|
|  0.0|4 Top Stock Trade...|[4, top, stock, t...|(11788,[5,8,61,17...|(11788,[5,

In [16]:
#Logistic Regression Classifier
#Cross validator different classification to  
#Param Grid apply different parameters, testing which param is best
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
lr = LogisticRegression(maxIter = 10)

paramGrid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, np.linspace(0.3, 0.01, 10)) \
    .addGrid(lr.elasticNetParam, np.linspace(0.3, 0.8, 6)) \
    .build()
crossval_lr = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid_lr,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds= 5)  
cvModel_lr = crossval_lr.fit(trainDF)
best_model_lr = cvModel_lr.bestModel.summary
best_model_lr.predictions.columns

['label',
 'text',
 'words',
 'rawFeatures',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [17]:
best_model_lr.predictions.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|                text|               words|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0.0|"ASSA ABLOY to ac...|["assa, abloy, to...|(11788,[0,2,4,313...|(11788,[0,2,4,313...|[1.64517033979983...|[0.79824527096704...|       0.0|
|  0.0|"BofA Unveils ""V...|["bofa, unveils, ...|(11788,[399,7222,...|(11788,[399,7222,...|[2.08263253514879...|[0.81353218445965...|       0.0|
|  0.0|"CP Offer for KCS...|["cp, offer, for,...|(11788,[5,75,344,...|(11788,[5,75,344,...|[1.76665185080529...|[0.80250194367115...|       0.0|
|  0.0|"Cumulus Podcast ...|["cumulus, podcas...|(11788,[0,2,10,13...|(11788,[0,2,10,13...|[2.90273855393937...|[0.82680486923848.

In [18]:
#import pre-covid test data
url1 = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/TEST_DATA_post-vax-test-data.csv'
spark.sparkContext.addFile(url1)
pre_covid_df = spark.read.csv(SparkFiles.get("TEST_DATA_post-vax-test-data.csv"), sep=",", header=None)

pre_covid_df.show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0                                                                                                                                                                                                                                                 |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ROSEN, TRUSTED AND LEADING INVESTOR COUNSEL, Reminds Royal Caribbean Cruises Ltd. Investors of Important December 7 Deadline in Securities Class Action; Encourages Investors with Losses in Excess of $100K to Contact Firm - RCL                  |
|ROSEN, GLOB

In [19]:
#Rename column
pre_covid_df2 = pre_covid_df.withColumnRenamed("_c0", "text")
pre_covid_df2.printSchema()
pre_covid_df2.show(20, truncate = False)

root
 |-- text: string (nullable = true)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ROSEN, TRUSTED AND LEADING INVESTOR COUNSEL, Reminds Royal Caribbean Cruises Ltd. Investors of Important December 7 Deadline in Securities Class Action; Encourages Investors with Losses in Excess of $100K to Contact 

In [20]:
#tokenize text
# from pyspark.ml.feature import  Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(pre_covid_df2)
wordsData.show()

+--------------------+--------------------+
|                text|               words|
+--------------------+--------------------+
|ROSEN, TRUSTED AN...|[rosen,, trusted,...|
|ROSEN, GLOBAL INV...|[rosen,, global, ...|
|ROSEN, GLOBAL INV...|[rosen,, global, ...|
|Hewlett Packard E...|[hewlett, packard...|
|BlackBerry to Hos...|[blackberry, to, ...|
|Alibaba Nears Dea...|[alibaba, nears, ...|
|Hyundai Motor to ...|[hyundai, motor, ...|
|Loral-Telesat Mer...|[loral-telesat, m...|
|Sally Beauty Hold...|[sally, beauty, h...|
|SoftBank winding ...|[softbank, windin...|
|Buy Netgear After...|[buy, netgear, af...|
|Meten EdtechX ann...|[meten, edtechx, ...|
|Fashion retailer ...|[fashion, retaile...|
|salesforce.com, i...|[salesforce.com,,...|
|Altimmune: A COVI...|[altimmune:, a, c...|
|Apple Sued For $2...|[apple, sued, for...|
|Box, Inc. (BOX) C...|[box,, inc., (box...|
|NetApp, Inc. (NTA...|[netapp,, inc., (...|
|Cogent Bioscience...|[cogent, bioscien...|
|BeiGene Announces...|[beigene, 

In [21]:
#count vectorize 
# from pyspark.ml.feature import CountVectorizer
# count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
# model = count.fit(wordsData)
featurizedData = model.transform(wordsData)
featurizedData.show()

+--------------------+--------------------+--------------------+
|                text|               words|         rawFeatures|
+--------------------+--------------------+--------------------+
|ROSEN, TRUSTED AN...|[rosen,, trusted,...|(11788,[0,2,3,4,1...|
|ROSEN, GLOBAL INV...|[rosen,, global, ...|(11788,[0,1,2,3,1...|
|ROSEN, GLOBAL INV...|[rosen,, global, ...|(11788,[0,1,2,3,1...|
|Hewlett Packard E...|[hewlett, packard...|(11788,[7,19,26,3...|
|BlackBerry to Hos...|[blackberry, to, ...|(11788,[0,22,44,1...|
|Alibaba Nears Dea...|[alibaba, nears, ...|(11788,[0,3,179,1...|
|Hyundai Motor to ...|[hyundai, motor, ...|(11788,[0,3,132,1...|
|Loral-Telesat Mer...|[loral-telesat, m...|(11788,[7,70,404,...|
|Sally Beauty Hold...|[sally, beauty, h...|(11788,[1,4,11,12...|
|SoftBank winding ...|[softbank, windin...|(11788,[44,65,115...|
|Buy Netgear After...|[buy, netgear, af...|(11788,[23,65,181...|
|Meten EdtechX ann...|[meten, edtechx, ...|(11788,[2,3,11,31...|
|Fashion retailer ...|[fa

In [22]:
from pyspark.ml.feature import  IDF
#IDF down-weighs features which appear frequently in a corpus. This generally improves performance when using text as features since most frequent, and hence less important words, get down-weighed.
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("features").show()  
# We want only the label and features columns for our machine learning models

+--------------------+
|            features|
+--------------------+
|(11788,[0,2,3,4,1...|
|(11788,[0,1,2,3,1...|
|(11788,[0,1,2,3,1...|
|(11788,[7,19,26,3...|
|(11788,[0,22,44,1...|
|(11788,[0,3,179,1...|
|(11788,[0,3,132,1...|
|(11788,[7,70,404,...|
|(11788,[1,4,11,12...|
|(11788,[44,65,115...|
|(11788,[23,65,181...|
|(11788,[2,3,11,31...|
|(11788,[45,391,12...|
|(11788,[7,13,14,1...|
|(11788,[6,65,181,...|
|(11788,[0,3,5,36,...|
|(11788,[7,13,14,1...|
|(11788,[7,13,14,1...|
|(11788,[2,8,11,41...|
|(11788,[2,11,75,9...|
+--------------------+
only showing top 20 rows



In [23]:
#Predict using the test data and evaluate the predictions
predictions_lr = cvModel_lr.transform(rescaledData)
predictions_lr.columns

['text',
 'words',
 'rawFeatures',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [24]:
predictions_lr.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|               words|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|ROSEN, TRUSTED AN...|[rosen,, trusted,...|(11788,[0,2,3,4,1...|(11788,[0,2,3,4,1...|[3.93304157614109...|[0.97771240476762...|       0.0|
|ROSEN, GLOBAL INV...|[rosen,, global, ...|(11788,[0,1,2,3,1...|(11788,[0,1,2,3,1...|[5.13326382791118...|[0.99502908192699...|       0.0|
|ROSEN, GLOBAL INV...|[rosen,, global, ...|(11788,[0,1,2,3,1...|(11788,[0,1,2,3,1...|[5.71481572304216...|[0.99883088236037...|       0.0|
|Hewlett Packard E...|[hewlett, packard...|(11788,[7,19,26,3...|(11788,[7,19,26,3...|[1.23440168659649...|[0.41800932381185...|       1.0|
|BlackBerry to Hos...|[blac

In [25]:
predictions_lr_df = predictions_lr.select('prediction', 'text','words')
predictions_lr_df.show(5)

+----------+--------------------+--------------------+
|prediction|                text|               words|
+----------+--------------------+--------------------+
|       0.0|ROSEN, TRUSTED AN...|[rosen,, trusted,...|
|       0.0|ROSEN, GLOBAL INV...|[rosen,, global, ...|
|       0.0|ROSEN, GLOBAL INV...|[rosen,, global, ...|
|       1.0|Hewlett Packard E...|[hewlett, packard...|
|       0.0|BlackBerry to Hos...|[blackberry, to, ...|
+----------+--------------------+--------------------+
only showing top 5 rows



In [26]:
predictions_lr_df.createOrReplaceTempView('temp')
predictions_lr_df1 = spark.sql('select case prediction when "1.0" then "Positive"  when "2.0" then "Negative" else "Neutral" end as predicted_label, text, words from temp')
predictions_lr_df1.show(20, truncate = False)

+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|predicted_label|text                                                                                                                                                                                                                                                |words                                                                                                                                                                                         

In [27]:
predictions_lr_df1.groupBy('predicted_label').count().show()

+---------------+-----+
|predicted_label|count|
+---------------+-----+
|        Neutral| 5932|
|       Positive| 2823|
|       Negative|  345|
+---------------+-----+



In [28]:
predictions_lr_df1.toPandas().to_csv('post-vax_predictions_count_vector_logistic_reg_model.csv')

In [29]:
#Naive Bayes using MulticlassClassificationEval
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
nb = NaiveBayes()
paramGrid_nb = ParamGridBuilder() \
    .addGrid(nb.smoothing, np.linspace(0.3, 10, 10)) \
    .build()
crossval_nb = CrossValidator(estimator=nb,
                          estimatorParamMaps=paramGrid_nb,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds= 5) 
cvModel_nb = crossval_nb.fit(trainDF)

In [30]:
#Make predictions:
predictions_nb = cvModel_nb.transform(rescaledData)
predictions_nb.select('text','prediction','words').show(truncate = False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                |prediction|words                                                                                                                                                                                                   

In [31]:
predictions_nb.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|               words|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|ROSEN, TRUSTED AN...|[rosen,, trusted,...|(11788,[0,2,3,4,1...|(11788,[0,2,3,4,1...|[-768.61721582493...|[1.0,1.1372679947...|       0.0|
|ROSEN, GLOBAL INV...|[rosen,, global, ...|(11788,[0,1,2,3,1...|(11788,[0,1,2,3,1...|[-914.48948346681...|[1.0,1.2491907025...|       0.0|
|ROSEN, GLOBAL INV...|[rosen,, global, ...|(11788,[0,1,2,3,1...|(11788,[0,1,2,3,1...|[-543.04913803859...|[1.0,2.9768056135...|       0.0|
|Hewlett Packard E...|[hewlett, packard...|(11788,[7,19,26,3...|(11788,[7,19,26,3...|[-467.80566663947...|[0.00164573678828...|       1.0|
|BlackBerry to Hos...|[blac

In [32]:
predictions_nb_df = predictions_nb.select('prediction', 'text','words')
predictions_nb_df.show(5)

+----------+--------------------+--------------------+
|prediction|                text|               words|
+----------+--------------------+--------------------+
|       0.0|ROSEN, TRUSTED AN...|[rosen,, trusted,...|
|       0.0|ROSEN, GLOBAL INV...|[rosen,, global, ...|
|       0.0|ROSEN, GLOBAL INV...|[rosen,, global, ...|
|       1.0|Hewlett Packard E...|[hewlett, packard...|
|       0.0|BlackBerry to Hos...|[blackberry, to, ...|
+----------+--------------------+--------------------+
only showing top 5 rows



In [33]:
predictions_nb_df.createOrReplaceTempView('temp')
predictions_nb_df1 = spark.sql('select case prediction when "1.0" then "Positive"  when "2.0" then "Negative" else "Neutral" end as predicted_label, text, words from temp')
predictions_nb_df1.show(20, truncate = False)

+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|predicted_label|text                                                                                                                                                                                                                                                |words                                                                                                                                                                                         

In [34]:
predictions_nb_df1.groupBy('predicted_label').count().show()

+---------------+-----+
|predicted_label|count|
+---------------+-----+
|        Neutral| 5314|
|       Positive| 3263|
|       Negative|  523|
+---------------+-----+



In [35]:
predictions_nb_df1.toPandas().to_csv('post-vax_predictions_count_vector_naive_bayes_model.csv')